<a href="https://colab.research.google.com/github/Flychuban/Tyres-Condition/blob/main/TyreCondition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import numpy as np
import glob
from pathlib import Path
import tensorflow as tf
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D

In [ ]:
train_path = Path('/content/drive/MyDrive/TyreConditionPrediction/data/Tire Textures/training_data')
test_path = Path('/content/drive/MyDrive/TyreConditionPrediction/data/Tire Textures/testing_data')
RANDOM_SEED = 10
EPOCHS = 10
BATCH_SIZE = 16

In [ ]:
labels = os.listdir(train_path)
labels

Let's make our dataset

In [ ]:
def process_data(data):
  path = Path(data)
  filepaths = list(path.glob(r'*/*.jpg'))
  labels=list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))#Separating the label from filepath and storing it
  df1 = pd.Series(filepaths, name='filepaths').astype(str)
  df2 = pd.Series(labels, name='labels')
  df = pd.concat([df1, df2], axis=1)
  return df

In [ ]:
train_data = process_data(train_path)
test_data = process_data(test_path)
train_data

In [ ]:
train_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_images = train_generator.flow_from_dataframe(dataframe=train_data,
                                                   target_size=(224, 224),
                                                   batch_size = BATCH_SIZE,
                                                   subset='training',
                                                   x_col='filepaths',
                                                   y_col='labels',
                                                   random_seed=RANDOM_SEED)

test_images = test_generator.flow_from_dataframe(dataframe=test_data,
                                                 target_size=(224, 224),
                                                 batch_size=BATCH_SIZE,
                                                 x_col='filepaths',
                                                 y_col = 'labels',
                                                 random_seed = RANDOM_SEED
                                                 )

Build the model

In [ ]:
pretrained_model = ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights = 'imagenet',
    pooling='max'
)

In [ ]:
pretrained_model.trainable = False # we make this cuz w don't train again the big model

In [ ]:
inputs = pretrained_model.input

new_layer = Dense(120, activation='relu')(pretrained_model.output)
new_layer = Dense(120, activation ='relu')(new_layer)

outputs = Dense(2, activation = 'softmax')(new_layer)

In [ ]:
model = Model(inputs=inputs, outputs=outputs)

In [ ]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
)

Training

In [ ]:
history = model.fit(train_images, epochs=EPOCHS)

In [ ]:
pd.DataFrame(history.history)[['accuracy']].plot()
plt.title("Accuracy")
plt.show()
pd.DataFrame(history.history)[['loss']].plot()
plt.title("Loss")
plt.show()

In [ ]:
model.evaluate(test_images)

In [ ]:
model.save("tyre_condition_model.h5")

In [3]:
model = tf.keras.models.load_model('/content/drive/MyDrive/TyreConditionPrediction/tyre_condition_model.h5')

In [35]:
from tensorflow.keras.preprocessing import image

test_image = image.load_img('/content/drive/MyDrive/TyreConditionPrediction/tyre4_img.jpg', target_size = (224, 224)) # Get the image
transformed_image = image.img_to_array(test_image) # Transform image to array
transformed_image = np.expand_dims(transformed_image, axis=0) # Expand dims to make a batch of 1 image
prediction = model.predict(transformed_image)

if prediction[0][0] > 0.5:
  print("This tyre HAS a problem!")
else:
  print("This tyre HASN'T problem!")

1/1 [==============================] - 0s 253ms/step
This tyre HASN'T problem!
